In [1]:
#!pip install gym
!pip install tensorflow[and-cuda]
# not compatible with tf >2.15
#!pip uninstall keras-rl2 -y

  Obtaining dependency information for tensorflow[and-cuda] from https://files.pythonhosted.org/packages/d7/24/f7b5c130975303efa4a91341e294fd784b94910c2a4c0f0f0561e5fc7405/tensorflow-2.17.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.17.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
Using cached tensorflow-2.17.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.4 MB)

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
#keras.__version__

In [49]:
print(tf.sysconfig.get_build_info()["cudnn_version"])

8


In [8]:
from gym import Env
from gym.spaces import Discrete, Box
import os

In [11]:
# set tf warning level to 2 ....shows errors but not warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
#tf.__version__

In [3]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model

import numpy as np
import random


2024-09-22 18:22:47.513611: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-22 18:22:47.523703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 18:22:47.533076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 18:22:47.535819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 18:22:47.542919: I tensorflow/core/platform/cpu_feature_guar

In [19]:
# Check if TensorFlow is built with CUDA support
if tf.config.list_physical_devices('GPU'):
    print("CUDA is available")
    print(f"Device: {tf.config.list_physical_devices('GPU')[0]}")
else:
    print("CUDA is not available")

CUDA is available
Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


# Create Environment

In [15]:
# inherit from gym env
class SwatEnv(Env):
    def __init__(self):
        # 0 decrease, 1 hold, 2 increase
        self.action_space = Discrete(3)
        # temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # set start temp
        self.state = 38 + random.randint(-3, 3)
        # set shower length
        self.shower_length = 60
        
    def step(self, action):
        # apply action
        # subtract 1 fro action so 1 - 1 = 0 no change
        self.state = self.state + action -1
        # reduce shower length
        self.shower_length = self.shower_length - 1
        
        # calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        # apply temp noise
        self.state += random.randint(-1,1)
        # place holder for info
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        # for vis ie pygame
        pass
    
    def reset(self):
        # reset temp
        self.state = 38 + random.randint(-3, 3)
        # reset shower time
        self.shower_length = 60
        return self.state
        
    

In [16]:
env = SwatEnv()

/home/loz/Documents/GitHub/MSc-Project/venv/lib/python3.12/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [22]:
# sample from action space
env.action_space.sample()

2

In [18]:
env.observation_space.sample()

array([25.64734], dtype=float32)

In [25]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:2
Episode:2 Score:-42
Episode:3 Score:-52
Episode:4 Score:-32
Episode:5 Score:-40
Episode:6 Score:-60
Episode:7 Score:-42
Episode:8 Score:-60
Episode:9 Score:-26
Episode:10 Score:-58


# Create Deep Learning Model#

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [27]:
states = env.observation_space.shape
actions = env.action_space.n

In [28]:
actions

3

In [32]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [36]:
#del model

In [37]:
model = build_model(states, actions)

In [38]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 24)             │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 24)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 723 (2.82 KB)

 Trainable params: 723 (2.82 KB)

 Non-trainable params: 0 (0.00 B)

# Build Agent wit Keras-RL

In [40]:
#! pip install keras-rl2

In [43]:
pip uninstall tensorflow -y

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
Note: you may need to restart the kernel to use updated packages.


In [45]:
!pip install tensorflow[and-cuda]==2.15

ERROR: Could not find a version that satisfies the requirement tensorflow[and-cuda]==2.15 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0)
ERROR: No matching distribution found for tensorflow[and-cuda]==2.15

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [41]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

ImportError: cannot import name 'model_from_config' from 'tensorflow.keras.models' (/home/loz/Documents/GitHub/MSc-Project/venv/lib/python3.12/site-packages/keras/_tf_keras/keras/models/__init__.py)

In [12]:
os.getcwd()

'/home/loz/Documents/GitHub/MSc-Project/final_code'

In [13]:
#os.listdir()

In [14]:
# get pre_trained model of SWaT
swat_lstm = tf.keras.models.load_model('swat_lstm_100s.keras')


I0000 00:00:1727014094.107138    6297 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727014094.107642    6297 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727014094.107869    6297 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727014094.187711    6297 cuda_executor.cc:1015] successful NUMA node read from SysFS ha